In [1]:
# Import the necessary libraries
import pandas as pd
import numpy as np
from PIL import Image
import PIL.ImageOps 
import glob
import matplotlib.pyplot as plt
%matplotlib inline
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
# Load the MNIST data provided by keras
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [35]:
# Reconstruct the Training images with the negative ones
# Turn them into numpy arrays
training_images = "mnist/train-images/*.jpg"
test_images = "mnist/test-images/*.jpg"
train_image_list = glob.glob(training_images)
test_image_list = glob.glob(test_images)

X_train =  np.array([np.array(Image.open(img)) for img in train_image_list])
X_test =  np.array([np.array(Image.open(img)) for img in test_image_list])

In [36]:
# The shape of each array is (60000, 28, 28)
# Since we are using tensorflow the format of each array should be (batch, height, width, channels)
# Let's reshape it then!
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train/=255
X_test/=255

In [37]:
# Apply one-hot encoding
number_of_classes = 10

Y_train = np_utils.to_categorical(y_train, number_of_classes)
Y_test = np_utils.to_categorical(y_test, number_of_classes)

In [38]:
# Now let's create the CNN model that will classify the MNIST images
# Steps:
# 1. Convolution
# 2. Activation
# 3. Pooling
# 4. Repetition of the previous steps to add more hidden layers
# 5. Create a fully connected network

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(28,28,1)))
BatchNormalization(axis=-1)
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
BatchNormalization(axis=-1)
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3, 3)))
BatchNormalization(axis=-1)
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
BatchNormalization(axis=-1)
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

# Fully connected layer
model.add(Dense(512))
BatchNormalization()
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))

model.add(Activation('softmax'))

In [39]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [40]:
# Apply data augmentation to reduce over-fitting
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                         height_shift_range=0.08, zoom_range=0.08)

test_gen = ImageDataGenerator()

In [41]:
# Create batches in order to use less memory.
# Using batch of 64, the model will take 64 images at a time in the process of training
train_generator = gen.flow(X_train, Y_train, batch_size=64)
test_generator = test_gen.flow(X_test, Y_test, batch_size=64)

In [42]:
# We're ready to train the model
# shuffle argument can be used here as well
model.fit_generator(train_generator, steps_per_epoch=60000//64, epochs=5, 
                    validation_data=test_generator, validation_steps=10000//64)

Epoch 1/5
937/937 [==============================] - 281s - loss: 2.3017 - acc: 0.1110 - val_loss: 2.3013 - val_acc: 0.1133
Epoch 2/5
937/937 [==============================] - 233s - loss: 2.3014 - acc: 0.1123 - val_loss: 2.3013 - val_acc: 0.1141
Epoch 3/5
937/937 [==============================] - 251s - loss: 2.3014 - acc: 0.1124 - val_loss: 2.3013 - val_acc: 0.1122
Epoch 4/5
937/937 [==============================] - 262s - loss: 2.3014 - acc: 0.1125 - val_loss: 2.3008 - val_acc: 0.1146
Epoch 5/5
937/937 [==============================] - 249s - loss: 2.3016 - acc: 0.1115 - val_loss: 2.3014 - val_acc: 0.1155
